In [19]:
import numpy as np
import torch

In [20]:
torch.__version__

'1.8.0+cu101'

In [5]:
torch.cuda.is_available()

False

1. Tensors (torch.Tensor, Tensor Attributes)

In [6]:
#from python list or sequence
a = [[1_000, 0], [3, 4]]
#dtype int8, unsigned int8, int16, int32, int64, float16, float32, float64, boolean
x = torch.tensor(a, dtype=torch.bool, device='cpu')
x.dtype, x.device, x.stride() #reference to tensor.Storage

(torch.bool, device(type='cpu'), (2, 1))

In [18]:
# 9 cpu constructors with specific dtype [torch.FloatTensor ...]
# 9 gpu constructors with specific dtype [torch.cuda.FloatTensor ...]

torch.float32

In [ ]:
# from array_like data
a = np.array([[1_000, 0], [3, 4]])
q = torch.as_tensor(a) #avoid copy
q[0, 0] = -1
a[0, 0], q.device
#on gpu this trick isn't work

(-1, device(type='cpu'))

In [ ]:
#from numpy (see bridge section)

In [16]:
#from another tensor
data = torch.tensor([[1, 2], [3, 4]])
x_ones = torch.ones_like(data)

x_rand = torch.rand_like(data, dtype=torch.float16)

x_ones, x_ones.dtype, x_rand, x_rand.dtype

(tensor([[1, 1],
         [1, 1]]), torch.int64, tensor([[0.7793, 0.8838],
         [0.1787, 0.9209]], dtype=torch.float16), torch.float16)

In [8]:
#numpy axis is equal to torch dim
a = np.array([[[1, 2, 3], 
               [3, 4, 0]],
                        [[1, 2, 3], 
                        [3, 4, 0]]])
a.shape, a.sum(axis=0), a.sum(axis=1), a.sum(axis=2)

((2, 2, 3), array([[2, 4, 6],
        [6, 8, 0]]), array([[4, 6, 3],
        [4, 6, 3]]), array([[6, 7],
        [6, 7]]))

In [10]:
b = torch.tensor(a)
b.sum(dim=0), b.sum(dim=1), b.sum(dim=2)

(tensor([[2, 4, 6],
         [6, 8, 0]]), tensor([[4, 6, 3],
         [4, 6, 3]]), tensor([[6, 7],
         [6, 7]]))

In [26]:
# tensor operations
# that have a _ suffix are in-place
#x.copy_(y)
#x.t_()
tensor = torch.ones((3, 3))
tensor.add_(5)
tensor

tensor([[6., 6., 6.],
        [6., 6., 6.],
        [6., 6., 6.]])

In [27]:
#numpy pytorch bridge
#Tensors on the CPU and NumPy arrays can share their underlying memory locations, and changing one will change the other.
  # torch -> numpy
t = torch.ones(5)
n = t.numpy()
t, n

(tensor([1., 1., 1., 1., 1.]), array([1., 1., 1., 1., 1.], dtype=float32))

In [28]:
t.add_(5)
t, n

(tensor([6., 6., 6., 6., 6.]), array([6., 6., 6., 6., 6.], dtype=float32))

In [36]:
  #numpy -> torch
n = np.ones(3)
t = torch.from_numpy(n)
n, t

(array([1., 1., 1.]), tensor([1., 1., 1.], dtype=torch.float64))

In [37]:
np.add(n, 1, out=n)
n, t

(array([2., 2., 2.]), tensor([2., 2., 2.], dtype=torch.float64))

2. torch.Autograd

In [ ]:
#torch autograd
t = torch.tensor([[1, 2, 3]], dtype=torch.float32, requires_grad=True)
out = t.pow(2).sum()
out.backward()
t.grad

tensor([[2., 4., 6.]])